In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

In [ ]:
import matplotlib.pyplot as plt
import numpy
import seaborn
import wordcloud

# Python Community Insights With Public Data

Using the the [PyPI public dataset](https://bigquery.cloud.google.com/dataset/the-psf:pypi) and the [StackOverflow public dataset](https://bigquery.cloud.google.com/dataset/bigquery-public-data:stackoverflow) what are some insights we can get about the Python community?

We'll be using Pandas and the Google Cloud client libraries.

In [ ]:
from google.cloud import bigquery
%load_ext google.cloud.bigquery

## What versions of Python are being used to download packages?

Using the the [PyPI public dataset](https://bigquery.cloud.google.com/dataset/the-psf:pypi), get a count of how many downloads there have been for each Python version in January 2018 so far.

In [ ]:
%%bigquery pypi_df

#standardSQL
SELECT
  REGEXP_EXTRACT(details.python, r"[0-9]+\.[0-9]+") AS python_version,
  COUNT(*) AS downloads
FROM `the-psf.pypi.downloads201801*`
WHERE details.installer.name = 'pip'
GROUP BY python_version
ORDER BY downloads DESC

In [ ]:
pypi_df.plot.bar('python_version', 'downloads')

We see that a significant portion of downloads are still using Python 2.7, but there is a visible portion using Python 3.x, with about half of those on Python 3.6.

## Is the portion using Python 3 changing?

In [ ]:
%%bigquery p3_df

#standardSQL
SELECT
  PARSE_DATE("%Y%m%d", _TABLE_SUFFIX) AS date,
  100 * COUNTIF(REGEXP_EXTRACT(details.python, r"^([^\.]+)") = "3") / COUNT(*) AS percent_3,
  COUNT(*) AS downloads
FROM `the-psf.pypi.downloads*`
WHERE details.installer.name = 'pip'
GROUP BY date

In [ ]:
# Convert to unix timestamp so we can plot.
# https://stackoverflow.com/a/8778548/101923

import datetime
from datetime import timezone

p3_df = p3_df.assign(
    date_num=p3_df['date'].apply(
        lambda d: datetime.datetime(
            d.year, d.month, d.day).replace(
            tzinfo=timezone.utc).timestamp()))

In [ ]:
p3_df.plot.scatter('date_num', 'percent_3')

For a deeper exploration of this dataset and how you can use it to learn about your favorite package, check out [Paul Kehrer's blog post](https://langui.sh/2016/12/09/data-driven-decisions/) and the [pypinfo CLI tool](https://github.com/ofek/pypinfo).

## What questions do people ask about Python?

Using the [StackOverflow public dataset](https://bigquery.cloud.google.com/dataset/bigquery-public-data:stackoverflow), what are the most popular questions people have about Python?

In [ ]:
%%bigquery so_df

#standardSQL
SELECT
  id, title, view_count
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags like '%python%'
ORDER BY view_count DESC
LIMIT 100

The most popular question is about how to [parse a string to a float or integer](https://stackoverflow.com/questions/379906/parse-string-to-float-or-int). All of these questions are about working with Python itself and not any specific package. Some top topics include:

- built-in objects like strings and dictionaries
- time
- parsing

Let's build a word cloud to see what topics pop out.

In [ ]:
wc = wordcloud.WordCloud(
    width=800, height=600, background_color='white')
wc.generate(' '.join(so_df.title))
plt.axis('off')
plt.imshow(wc, interpolation='bilinear')

## Are some packages harder to use?

The nice thing about BigQuery is that if you have access to several datasets, you can join them together. Even though the PyPI dataset is in `the-psf` project and the StackOverlow dataset is in the `bigquery-public-data` project, we can still join them together.

I'm interested to know which packages have more questions per download in 2017.

In [ ]:
%%bigquery qpd_df

#standardSQL
SELECT
  project,
  SUM(questions) / downloads AS qpd,
  SUM(questions) AS total_questions,
  downloads
FROM (
  SELECT
    file.project AS project,
    COUNT(*) AS downloads
  FROM `the-psf.pypi.downloads2017*`
  WHERE details.installer.name = 'pip'
  GROUP BY project ) pypi
JOIN (
  SELECT
    tags,
    COUNT(*) AS questions
  FROM `bigquery-public-data.stackoverflow.posts_questions`
  WHERE tags LIKE '%python%'
  GROUP BY tags ) so
ON pypi.project IN UNNEST(SPLIT(so.tags, '|'))
WHERE pypi.downloads >= 10000
GROUP BY project, downloads
ORDER BY qpd DESC

Some of these, like the [list package](https://pypi.org/project/List/1.3.0/) do exist, but refer to built-in modules / types and were likely downloaded by mistake.

Others, like `opencv` and `boost` really are pretty complex packages, so I'm not too surprised that there are more questions about them.

In [ ]:
qpd_df.plot.scatter('downloads', 'total_questions')

There isn't an obvious trend, but we see that our query has grabbed some of those outliers with many questions and relatively few downloads.